# Testing functionality of FAQ class 
Below is the actual code for the FAQ algorithm. 
This code is an implementation of Algorithm 1 from "Fast Approximate Quadratic Programming
for Graph Matching" (Vogelstein, 2015) --> https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0121002&type=printable.





In [125]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
from sinkhorn_knopp import sinkhorn_knopp as skp
sk = skp.SinkhornKnopp()

class FAQ():
    '''
    Fast Approximate QAP Algorithm (FAQ)
    The FAQ algorithm solves the Quadratic Assignment Problem (QAP) finding an alignment of the
    vertices of two graphs which minimizes the number of induced edge disagreements
    
    Parameters
    ----------
    
    n_init : int
        Number of random initializations of the starting permutation matrix that the FAQ algorithm will undergo
        
    init_method : string
        The intial position chosen
        "barycenter" : he noninformative “flat doubly stochastic matrix,” J=1*1^T/n, i.e the barycenter of 
        the feasible region
        "rand" : some random point near J, (J+K)/2, where K is some random doubly stochastic matrix
        
    Attributes
    ----------
    
    perm_inds_ : array, size (n,1) where n is the number of vertices in the graphs fitted
        The indices of the optimal permutation found via FAQ
        
    ofv_ : float
        The objective function value of for the optimal permuation found
        
        
    ''' 
    def __init__(self, n_init, init_method, ):
        
        if n_init > 0 and type(n_init) is int:
            self.n_init = n_init
        else:
            msg = '"n_init" must be a positive integer'
            raise TypeError(msg)
        if init_method == 'rand':
            self.init_method = 'rand'
        elif init_method == "barycenter":
            self.init_method = 'barycenter'
            n_init = 1
        else: 
            msg = 'Invalid "init_method" parameter string'
            raise ValueError(msg)
    def fit(self, A, B):
        '''
        Fits the model with two assigned adjacency matrices
        
        Parameters
        ---------
        A : 2d-array, square
            A square adjacency matrix
            
        B : 2d-array, square
            A square adjacency matrix
        
        Returns
        -------
        
        self : returns an instance of self
        '''
        if A.shape[0] != B.shape[0]:
            msg = "Matrix entries must be of equal size"
            raise ValueError(msg)
        elif ((A>=0).all()==False or (B>=0).all() == False):
            msg = "Matrix entries must be greater than or equal to zero"
            raise ValueError(msg)
        else:
        
            n = A.shape[0] #number of vertices in graphs
            At = np.transpose(A)  # A transpose
            Bt = np.transpose(B)  # B transpose
            ofv_ = math.inf 
            perm_inds_ = np.zeros(n)

            for i in range(self.n_init):

                #setting initialization matrix
                if self.init_method == 'rand': 
                    sk = skp.SinkhornKnopp() 
                    K = np.random.rand(n,n) #generate a nxn matrix where each entry is a random integer [0,1]
                    for i in range(10): #perform 10 iterations of Sinkhorn balancing
                        K = sk.fit(K)
                    J = np.ones((n,n))/float(n) # initialize J, a doubly stochastic barycenter
                    P = (K+J)/2
                elif self.init_method == 'barycenter':
                    P = np.ones((n,n))/float(n)

                #OPTIMIZATION WHILE LOOP BEGINS
                for i in range(30):

                    delfp = A@P@Bt+At@P@B  # computing the gradient of f(P) = -tr(APB^tP^t)
                    rows, cols = linear_sum_assignment(delfp) # run hungarian algorithm on gradient(f(P))
                    Q = np.zeros((n,n))  
                    Q[rows,cols] = 1   # initialize search direction matrix Q

                    def f(x):  #computing the original optimization function
                        return np.trace(At@(x*P+(1-x)*Q)@B@np.transpose(x*P+(1-x)*Q))

                    alpha = minimize_scalar(f, bounds=(0,1), method='bounded').x #computing the step size
                    P = alpha*P + (1-alpha)*Q  # Update P
                #end of FW optimization loop

                row, perm_inds_new = linear_sum_assignment(-P) #Project onto the set of permutation matrices
                perm_mat_new = np.zeros((n,n)) #initiate a nxn matrix of zeros
                perm_mat_new[row,perm_inds_new] = 1 # set indices of permutation to 1
                ofv_new = np.trace(np.transpose(A)@perm_mat_new@B@np.transpose(perm_mat_new)) #computing objective function value

                if ofv_new < ofv_: #minimizing
                    ofv_ = ofv_new
                    perm_inds_ = perm_inds_new

            self.perm_inds_ = perm_inds_  # permutation indices
            self.ofv_ = ofv_  # objective function value
            return self


# QAP Benchmark Accuracy
To test that the function works, I ran the code using data from QAPLIB, an online database testbed of QAP data. The tests were run using a flat doubly stochastic initialization (barycenter) for reproducability and consistency. The results (objective function values (ofv)) of the test were compared against the optimal values reported on the QAPLIB website, as well as the values reported in Table 1 of FAQ (Vogelstein, 2015), which were run "on a set of 16 particularly difficult directed benchmarks from QAPLIB." 

In [80]:
#faqali = []
#problems = []

with open('qapdata/lipa20a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa20a = barycenter.fit(A,B)
faqali.append(lipa20a.ofv_)
problems.append('lipa20a')

print(lipa20a.ofv_)

3768.0


In [47]:
with open('qapdata/lipa20b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa20b = barycenter.fit(A,B)
faqali.append(lipa20b.ofv_)
problems.append('lipa20b')
print(lipa20b.ofv_)

27076.0


In [48]:
with open('qapdata/lipa30a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa30a = barycenter.fit(A,B)
faqali.append(lipa30a.ofv_)
problems.append('lipa30a')
print(lipa30a.ofv_)

13451.0


In [50]:
with open('qapdata/lipa30b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa30b = barycenter.fit(A,B)

faqali.append(lipa30b.ofv_)
problems.append('lipa30b')
print(lipa30b.ofv_)

151426.0


In [51]:
with open('qapdata/lipa40a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa40a = barycenter.fit(A,B)

faqali.append(lipa40a.ofv_)
problems.append('lipa40a')
print(lipa40a.ofv_)

32101.0


In [52]:
with open('qapdata/lipa40b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa40b = barycenter.fit(A,B)

faqali.append(lipa40b.ofv_)
problems.append('lipa40b')
print(lipa40b.ofv_)

476581.0


In [53]:
with open('qapdata/lipa50a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa50a = barycenter.fit(A,B)

faqali.append(lipa50a.ofv_)
problems.append('lipa50a')
print(lipa50a.ofv_)

63084.0


In [54]:
with open('qapdata/lipa50b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa50b = barycenter.fit(A,B)

faqali.append(lipa50b.ofv_)
problems.append('lipa50b')
print(lipa50b.ofv_)

1210244.0


In [55]:
with open('qapdata/lipa60a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa60a = barycenter.fit(A,B)

faqali.append(lipa60a.ofv_)
problems.append('lipa60a')
print(lipa60a.ofv_)

108712.0


In [56]:
with open('qapdata/lipa60b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa60b = barycenter.fit(A,B)

faqali.append(lipa60b.ofv_)
problems.append('lipa60b')
print(lipa60b.ofv_)

2520135.0


In [57]:
with open('qapdata/lipa70a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa70a = barycenter.fit(A,B)

faqali.append(lipa70a.ofv_)
problems.append('lipa70a')
print(lipa70a.ofv_)

171963.0


In [58]:
with open('qapdata/lipa70b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa70b = barycenter.fit(A,B)

faqali.append(lipa70b.ofv_)
problems.append('lipa70b')
print(lipa70b.ofv_)

4603200.0


In [59]:
with open('qapdata/lipa80a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa80a = barycenter.fit(A,B)

faqali.append(lipa80a.ofv_)
problems.append('lipa80a')
print(lipa80a.ofv_)

255978.0


In [60]:
with open('qapdata/lipa80b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa80b = barycenter.fit(A,B)

faqali.append(lipa80b.ofv_)
problems.append('lipa80b')
print(lipa80b.ofv_)

7763962.0


In [61]:
with open('qapdata/lipa90a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa90a = barycenter.fit(A,B)

faqali.append(lipa90a.ofv_)
problems.append('lipa90a')
print(lipa90a.ofv_)

364121.0


In [62]:
with open('qapdata/lipa90b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa90b = barycenter.fit(A,B)

faqali.append(lipa90b.ofv_)
problems.append('lipa90b')
print(lipa90b.ofv_)

12490441.0


In [63]:
print(faqali)
print(problems)

[3768.0, 27076.0, 13451.0, 151426.0, 32101.0, 476581.0, 63084.0, 1210244.0, 108712.0, 2520135.0, 171963.0, 4603200.0, 255978.0, 7763962.0, 364121.0, 12490441.0]
['lipa20a', 'lipa20b', 'lipa30a', 'lipa30b', 'lipa40a', 'lipa40b', 'lipa50a', 'lipa50b', 'lipa60a', 'lipa60b', 'lipa70a', 'lipa70b', 'lipa80a', 'lipa80b', 'lipa90a', 'lipa90b']


In [70]:
optimal = [3683,27076,13178,151426,31538,476581,62093,1210244,107218,2520135,
          169755,4603200,253195,7763962,360630,12490441]
faq_paper = [3791,27076,13571,151426,32109,476581,62962,1210244,108488,2520135,
            171820,4603200,256073,7763962,363927,12490441]
faq_jovo_code = [3851,27076,13618,151426,32190,476581,63325,1210244,109085, 2520135, 
                172301,4603200, 256797,7763962,365080,12490441]

In [71]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(header=dict(values=['Problem','Optimal','FAQ (JOVO Paper)','FAQ (JOVO Code)','FAQ (ALI)']),
        cells = dict(values=[problems,optimal,faq_paper,faq_jovo_code, faqali]))])
fig.show()
                               
                               

# Results
As we can see from the values listed above, for problems where jovo's FAQ values reported in the paper reach the optimal value, so do mine. However, due to slight discrepancy between the other values, I additionally compared the results from running jovo's matlab code. These values also have a light discrepancy between those reported in the paper. Since my values are consistently less than or equal to the values from jovo's code and paper, and always greater than or equal to the optimal value, I believe this proves the accuracy of my implementation

# Multiple Restarts
Since the feasible region has many local minima, "our FAQ procedure is sensitive to the chosen initial position. With this in mind, we propose a variant of the FAQ algorithm in which we run the FAQ procedure with
multiple initializations. The algorithm outputs the best FAQ iterate over all the initializations.
For each initialization, we sample K 2 D, a random doubly stochastic matrix, using 10 iterations of Sinkhorn balancing, and let our initialization be P(0) = (J+K)/2, where J is the doubly stochastic barycenter." 
"Note that we only consider the 16 particularly difficult
benchmarks for this evaluation."

In [106]:
#mult_ali = []
#mult_problems = []

with open('qapdata/chr12c.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
rand_100_inits = FAQ(100,"rand")
chr12c = rand_100_inits.fit(A,B)

print(chr12c.ofv_)
mult_ali.append(chr12c.ofv_)
mult_problems.append('chr12c')

11914.0


In [107]:
with open('qapdata/chr15a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

chr15a = rand_100_inits.fit(A,B)
print(chr15a.ofv_)
mult_ali.append(chr15a.ofv_)
mult_problems.append('chr15a')

10440.0


In [108]:
with open('qapdata/chr15c.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

chr15c = rand_100_inits.fit(A,B)
print(chr15c.ofv_)
mult_ali.append(chr15c.ofv_)
mult_problems.append('chr15c')

11136.0


In [109]:
with open('qapdata/chr20b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

chr20b = rand_100_inits.fit(A,B)
print(chr20b.ofv_)
mult_ali.append(chr20b.ofv_)
mult_problems.append('chr20b')

2756.0


In [110]:
with open('qapdata/chr22b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

chr22b = rand_100_inits.fit(A,B)
print(chr22b.ofv_)
mult_ali.append(chr22b.ofv_)
mult_problems.append('chr22b')

6988.0


In [111]:
with open('qapdata/esc16b.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

esc16b = rand_100_inits.fit(A,B)
print(esc16b.ofv_)
mult_ali.append(esc16b.ofv_)
mult_problems.append('esc16b')

292.0


In [112]:
with open('qapdata/rou12.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

rou12 = rand_100_inits.fit(A,B)
print(rou12.ofv_)
mult_ali.append(rou12.ofv_)
mult_problems.append('rou12')

235528.0


In [113]:
with open('qapdata/rou15.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

rou15 = rand_100_inits.fit(A,B)
print(rou15.ofv_)
mult_ali.append(rou15.ofv_)
mult_problems.append('rou15')

354210.0


In [114]:
with open('qapdata/rou20.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

rou20 = rand_100_inits.fit(A,B)
print(rou20.ofv_)
mult_ali.append(rou20.ofv_)
mult_problems.append('rou20')

728448.0


In [115]:
with open('qapdata/tai10a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

tai10a = rand_100_inits.fit(A,B)
print(tai10a.ofv_)
mult_ali.append(tai10a.ofv_)
mult_problems.append('tai10a')

135028.0


In [116]:
with open('qapdata/tai15a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

tai15a = rand_100_inits.fit(A,B)
print(tai15a.ofv_)
mult_ali.append(tai15a.ofv_)
mult_problems.append('tai15a')

390258.0


In [117]:
with open('qapdata/tai17a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

tai17a = rand_100_inits.fit(A,B)
print(tai17a.ofv_)
mult_ali.append(tai17a.ofv_)
mult_problems.append('tai17a')

496598.0


In [118]:
with open('qapdata/tai20a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

tai20a = rand_100_inits.fit(A,B)
print(tai20a.ofv_)
mult_ali.append(tai20a.ofv_)
mult_problems.append('tai20a')

708198.0


In [119]:
with open('qapdata/tai30a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

tai30a = rand_100_inits.fit(A,B)
print(tai30a.ofv_)
mult_ali.append(tai30a.ofv_)
mult_problems.append('tai30a')

1828454.0


In [120]:
with open('qapdata/tai35a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

tai35a = rand_100_inits.fit(A,B)
print(tai35a.ofv_)
mult_ali.append(tai35a.ofv_)
mult_problems.append('tai35a')

2461978.0


In [121]:
with open('qapdata/tai40a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]

tai40a = rand_100_inits.fit(A,B)
print(tai40a.ofv_)
mult_ali.append(tai40a.ofv_)
mult_problems.append('tai40a')

3181708.0


In [122]:

print(mult_ali)

[11914.0, 10440.0, 11136.0, 2756.0, 6988.0, 292.0, 235528.0, 354210.0, 728448.0, 135028.0, 390258.0, 496598.0, 708198.0, 1828454.0, 2461978.0, 3181708.0]


In [123]:
optimal_mr = [11156,9896,9504,2298,6194,292,235528,354210,725522,135028,388214,491812,703482,1818146,2422002,3139370]
jovo_mr = [12176,9896,10960,2786,7218,292,235528,256654,730614,135828,391522,496598,711840,1844636,2454292,3187738]



import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(header=dict(values=['Problem','Optimal','FAQ (JOVO Paper)','FAQ (ALI)']),
        cells = dict(values=[mult_problems,optimal,jovo_mr,mult_ali]))])
fig.show()

# For now, ignore everything below this point

Additionally, I decided another good way to show the performance would be to test how the algorithm solves the graph matching problem. "The objective function for GM is just the negative of the objective function for QAP." The adjusted code to solve the GMP is found below

Below is the test code to check whether or not the GMP can be successfully solved. Here I am synthesizing data. First, a random nxn symmetric permutation matrix is instantiated, which will be the adjacency matrix A. Then, some random permutation to A is generated (a randomly shuffled list [0, n-1]), which is then converted to a permutation matrix (P_opt). Adjancency matrix B is generated through the equation B = (P_opt)^T * A * P_opt. Now we have some adjancy matrix A and some random permutation of A, B. To test the performance accuracy of the function, 50 random instances of A will be instantiated. If there are zero edge disagreement between A and the permutation of B return by FAQ (P), then trace(AP-PB) = 0. This test will be run for n = 8,10,12,14,16 with the success rate returned.

In [ ]:
def GMPtest(n):
    correct = 0
    for i in range(50):
    
        def choose(n, k):
            """
            A fast way to calculate binomial coefficients by Andrew Dalke (contrib).
            """
            if 0 <= k <= n:
                ntok = 1
                ktok = 1
                for t in range(1, min(k, n - k) + 1):
                    ntok *= n
                    ktok *= t
                    n -= 1
                return ntok // ktok
            else:
                return 0
        tri_size = choose(n,2)
        arr = np.random.randint(2,size = tri_size)
        A = np.zeros((n,n))
        iter_ = 0
        for i in range(n):
            for j in range(i+1,n):
                A[i,j] = arr[iter_]
                A[j,i] = arr[iter_]
                iter_ += 1
        row = np.arange(n)
        col = np.arange(n)
        np.random.shuffle(col)
        P_opt = np.zeros((n,n))
        P_opt[row,col] = 1
        B = np.transpose(P_opt)@A@P_opt

        #print(arr)
        #print(A)
        #print(P_opt)
        #print(B)

        #print(row)
        #print(col)
        #print(sum(sum(A)))
        #print(sum(sum(B)))

        rand_150_inits = FAQ(100,"rand")
        test = rand_150_inits.fit(A,B)
        #print(test.perm_inds_)
        #print(test.ofv_)

        #print(np.trace(A@P_opt@np.transpose(B)@np.transpose(P_opt)))
        P = np.zeros((n,n))
        P[row, test.perm_inds_] = 1
        #print(A@P-P@B)
        if np.linalg.norm(A@P-P@B, ord='fro') == 0:
            correct += 1
    return correct/100.0

            


In [ ]:
GMPtest(8)

In [ ]:
print(GMPtest(10))

In [ ]:
print(GMPtest(12))

In [ ]:
print(GMPtest(14))

In [ ]:
print(GMPtest(16))

As we can see from the above cells, running with graphs of size 8,10,12,14,16 vertices all yield 100% accuracy

In [26]:
with open('qapdata/lipa20a.dat') as f:
        f = [int(elem) for elem in f.read().split()]

# adjusting
f = np.array(f[1:])
n = int(math.sqrt(len(f)/2))
f = f.reshape(2*n,n)
A = f[:n,:]
B = f[n:,:]
barycenter = FAQ(2,"barycenter")
lipa20a = barycenter.fit(A,B)

print(lipa20a.ofv_)


3768.0


In [8]:
At = np.transpose(A)  # A transpose
Bt = np.transpose(B)
P = np.ones((n,n))/float(n)

In [9]:
delfp = (A@P@Bt+At@P@B)

In [11]:
rows, cols = linear_sum_assignment(delfp) # run hungarian algorithm on gradient(f(P))
Q = np.zeros((n,n))  
Q[rows,cols] = 1
print(Q)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.

In [12]:
np.trace(np.transpose(delfp)@Q)

7473.800000000001

In [ ]:
cols = [11,0,6,14,2,3,10,9,7,13,15,4,16,18,17,8,19,1,12,5]
Q = np.zeros((n,n))  
Q[rows,cols] = 1
np.trace(np.transpose(delfp)@Q)

In [ ]:
rows, cols = linear_sum_assignment(-delfp) # run hungarian algorithm on gradient(f(P))
Q = np.zeros((n,n))  
Q[rows,cols] = 1
print(Q)

In [ ]:
np.trace(np.transpose(delfp)@Q)

In [13]:
cols = [11,0,6,14,2,3,10,9,7,13,15,4,16,18,17,8,19,1,12,5]
Q = np.zeros((n,n))  
Q[rows,cols] = 1
np.trace(np.transpose(delfp)@Q)

7473.800000000001

In [ ]:
def f(x):  #computing the original optimization function
    return -np.trace(At@(x*P+(1-x)*Q)@B@np.transpose(x*P+(1-x)*Q))

alpha = minimize_scalar(f, bounds=(0,1), method='bounded').x #computing the step size
P = alpha*P + (1-alpha)*Q  # Update P